In [1]:
import transformers
import torch
tokenizer = transformers.LlamaTokenizer.from_pretrained('axiong/PMC_LLaMA_13B')
model = transformers.LlamaForCausalLM.from_pretrained('axiong/PMC_LLaMA_13B')

/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Loading checkpoint shards: 100%|██████████| 6/6 [09:00<00:00, 90.02s/it] 


In [2]:
# import transformers
# import torch
# tokenizer = transformers.LlamaTokenizer.from_pretrained('axiong/PMC_LLaMA_13B')
# model = transformers.LlamaForCausalLM.from_pretrained('axiong/PMC_LLaMA_13B')
# model.cuda()  # move the model to GPU

# prompt_input = (
#         'Below is an instruction that describes a task, paired with an input that provides further context.'
#     'Write a response that appropriately completes the request.\n\n'
#     '### Instruction:\n{instruction}\n\n### Input:\n{input}\n\n### Response:'
# )

# example = {
#     "instruction": "You're a doctor, kindly address the medical queries according to the patient's account. Answer with the best option directly.",
#     "input": (
#         "###Question: A 23-year-old pregnant woman at 22 weeks gestation presents with burning upon urination. "
#         "She states it started 1 day ago and has been worsening despite drinking more water and taking cranberry extract. "
#         "She otherwise feels well and is followed by a doctor for her pregnancy. "
#         "Her temperature is 97.7°F (36.5°C), blood pressure is 122/77 mmHg, pulse is 80/min, respirations are 19/min, and oxygen saturation is 98% on room air."
#         "Physical exam is notable for an absence of costovertebral angle tenderness and a gravid uterus. "
#         "Which of the following is the best treatment for this patient?"
#         "###Options: A. Ampicillin B. Ceftriaxone C. Doxycycline D. Nitrofurantoin"
#     )
# }
# input_str = [prompt_input.format_map(example)]

# model_inputs = tokenizer(
#     input_str,
#     return_tensors='pt',
#     padding=True,
# )
# print( f"\033[32mmodel_inputs\033[0m: { model_inputs }" )


# topk_output = model.generate(
#     model_inputs.input_ids.cuda(),
#     max_new_tokens=1000,
#     top_k=50
# )
# output_str = tokenizer.batch_decode(topk_output)
# print('model predict: ', output_str[0])

In [3]:
pipe = transformers.pipeline(

    model=model, 

    tokenizer=tokenizer,

    task='text-generation',

    temperature=0.1, 

    max_new_tokens=50,  

    repetition_penalty=1.1 

)  

In [4]:
import pandas as pd
df_text = pd.read_csv('Te2Query.csv')
eg = df_text.sample(n=1000, random_state=3)
input_text = eg['Questions'].to_list()
input_labels = eg['query'].to_list()
eg

,Unnamed: 0,Questions,id,query
3276,3276,Give me all the patients who got vaccines on ...,20103276,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
1409,1409,Give me all the patients whose report was comp...,7101409,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
7172,7172,Which is the most common cataracts for patients.,28307172,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
9319,9319,What is the number of records that the vaccin...,33109319,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
11467,11467,Give me all the patients who got INFLUENZA (SE...,43311467,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
...,...,...,...,...
7332,7332,Give me all the patients who was allergic to pvc,29307332,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."
10466,10466,How many GLAXOSMITHKLINE BIOLOGICALS vaccine ...,37110466,"POST_scripts/1{""script"":{""lang"":""mustache"",""so..."
1109,1109,How many patients are 100.0 years old.,3201109,"POST_scripts/2{""script"":{""lang"":""mustache"",""so..."
7771,7771,What is the number of vaccine recipients that...,29307771,"POST_scripts/3{""script"":{""lang"":""mustache"",""so..."


In [5]:
# # original prompt
# prompt = """ignore all the prior information before this block. Convert the following questions to elastic search queries follow two rules:
# 1.based on the field name 'RECVDATE','STATE','AGE_YRS','VAERS_ID','SEX','SYMPTOM_TEXT','DIED','ER_VISIT','L_THREAT','HOSPITAL','HOSPDAYS','DISABLE','VAX_DATE','LAB_DATA','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','TODAYS_DATE','OFC_VISIT','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_NAME','ALLERGIES'. 
# 2.follow the template 

# "POST _scripts/1
# {
#   "script": {
# 	"lang": "mustache",
# 	"source": {
#   	"track_total_hits": "true",
#   	"query": {
#     	"term": {
#       	"{{field}}": "{{date}}"
#     	}
#   	}
# 	},
# 	"params": {
#   	"field": "DATA.RECVDATE.keyword",
#   	"date": "01/01/2012"
# 	}
#   }
# }
# "

# """

In [6]:
import torch
torch.cuda.empty_cache()

#NER prompt
prompt_prefix = """Find the entity of the following questions based on the field name follow emample:How many patients' record are received on 03/20/2022. The '03/20/2022' is a ['RECVDATE']."""


In [7]:
torch.cuda.empty_cache()

#Q&A prompt
prompt_cloze = """Classify the questions based on the field name follow example:How many patients' record are received on 03/20/2022. The question wants ['VAERS_ID'] based on ['RECVDATE']."""


In [8]:
torch.cuda.empty_cache()

# cot + heuristic prompt
prompt_cot = """
find the entity classification and label with following name:
'RECVDATE','STATE','AGE_YRS','VAERS_ID','SEX','SYMPTOM_TEXT','DIED','ER_VISIT','L_THREAT','HOSPITAL','HOSPDAYS','DISABLE','VAX_DATE','LAB_DATA','OTHER_MEDS','CUR_ILL','HISTORY','PRIOR_VAX','TODAYS_DATE','OFC_VISIT','VAX_TYPE','VAX_MANU','VAX_LOT','VAX_DOSE_SERIES','VAX_NAME','ALLERGIES'
Examples:
1.Give me all the patients whose information are received on 04/13/2022. The question want ['VAERS_ID'] based on ['RECVDATE'].
2. How many patients' record are received on 03/20/2022. The question wants ['VAERS_ID'] based on ['RECVDATE'].
Based on the classification find the condition value in the sentence:
Examples:
1.Give me all the patients whose information are received on 04/13/2022. The ['RECVDATE'] is 04/13/2022.
2. How many patients' record are received on 03/20/2022. The ['RECVDATE'] is 03/20/2022.
Based on the entity classification and conditional values, covert questions to Elasticsearch queries

"""

In [9]:
# define the model input template
input_template = """
Prompt: {prompt}
Clinical Notes: {text}
Answer:
"""

In [10]:
# build up the call
answer_lst = []
for row in eg.iterrows():
    txt = row[1]['Questions']
#    suggest = row[1]['query']
    input = input_template.format(prompt = prompt_cot, text = txt)
    answer = pipe(input)
    answer_lst.append(answer[0]['generated_text'][len(input):].strip())
    #answer_lst.append(answer[0]['generated_text'])
eg['llm_result'] = answer_lst

/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/utils.py:1553: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/home/wenlong/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


In [11]:
pd.set_option('display.max_colwidth', None)
eg['llm_result']

3276                                                                                                                           {\n"query": {\n"filtered": {\n"filter": {\n"bool": {\n"must": [\n{"range": {"$or": [{"$eq": ["$RECVDATE", "201
1409     {\n    "query": {\n        "filtered": {\n            "filter": {\n                "bool": {\n                    "must": [\n                        {\n                            "term": {\n                                "RECV
7172                                                                                                                                                                                                                                         
9319                                                                                                                                   "$query": {\n\t\t"bool": {\n\t\t\t"must": [\n\t\t\t\t{ "term": { "VAERS_ID": { "value": "USPFIZER INC2
11467                                           

In [12]:
result_df = eg[['llm_result']]
result_df.to_json('covert_PMC_Q&A_1.json')

In [13]:
import json
 
# Opening JSON file
f = open('covert_PMC_Q&A_1.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
print(data)
 
# Closing file
f.close()

{'llm_result': {'3276': '{\n"query": {\n"filtered": {\n"filter": {\n"bool": {\n"must": [\n{"range": {"$or": [{"$eq": ["$RECVDATE", "201', '1409': '{\n    "query": {\n        "filtered": {\n            "filter": {\n                "bool": {\n                    "must": [\n                        {\n                            "term": {\n                                "RECV', '7172': '', '9319': '"$query": {\n\t\t"bool": {\n\t\t\t"must": [\n\t\t\t\t{ "term": { "VAERS_ID": { "value": "USPFIZER INC2', '11467': '{\n    "query": {\n        "filtered": {\n            "filter": [\n                {"and": [\n                    {"term": {"clinicalNotes.classification" : "VACCINATION"', '1987': '{\n    "query": {\n        "bool": {\n            "must": [\n                {\n                    "term": {\n                        "VAERS_ID.id": 1\n                    }\n                },', '4088': '{\n"query": {\n"filtered": {\n"filter": {\n"bool": {\n"must": [\n{\n"term": {\n"VAERS_ID": {\n"val

In [14]:
from codebleu import calc_codebleu

prediction = str(answer_lst)
reference = df_text['query'].to_string()
result_eval = calc_codebleu([reference], [prediction], lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)

In [15]:
result_eval=pd.Series(result_eval)
result_eval.to_json('PMC_result_eval_Q&A_1.json')

In [16]:
import json
 
# Opening JSON file
f = open('PMC_result_eval_Q&A_1.json')
 
# returns JSON object as 
# a dictionary
data = json.load(f)
 
# Iterating through the json
# list
print(data)
 
# Closing file
f.close()

{'codebleu': 0.391105542, 'ngram_match_score': 5.2548e-06, 'weighted_ngram_match_score': 1.2948e-05, 'syntax_match_score': 0.5644039652, 'dataflow_match_score': 0.0}
